In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database
import myLogReader as mlr
import re
import os
import sys
import datetime as dt

%matplotlib inline

## Automate - Read and Prep log data into DF

In [2]:
logsPath = '../data/logs'
geoLiteIPDBPath = '../data/GeoLite2-City_20181009/GeoLite2-City.mmdb'

#Create a myLogReader object
myLogReader = mlr.log()
#Open Reader
myLogReader.openReader(geoLiteIPDBPath)

In [3]:
df =  myLogReader.readLogs(logsPath,1)

../data/logs\u_ex171208.log


In [4]:
#Close Reader
myLogReader.closeReader()

In [5]:
df.head()

,date,cs-username,client-ip,time-taken(ms),client-device,client-browser,client-webPage,client-city,client-country,date-IsWeekday,date-calendar-week,date-year
0,2017-12-08,NaN,144.139.133.243,140,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
1,2017-12-08,NaN,144.139.133.243,46,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
2,2017-12-08,NaN,144.139.133.243,78,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
3,2017-12-08,NaN,144.139.133.243,171,Desktop,Firefox,Security,Melbourne,Australia,1,49,2017
4,2017-12-08,NaN,144.139.133.243,46,Desktop,Firefox,Security,Melbourne,Australia,1,49,2017


In [14]:
df['cs-username'].fillna('Unknown',inplace=True)

In [15]:
df.head()

,date,cs-username,client-ip,time-taken(ms),client-device,client-browser,client-webPage,client-city,client-country,date-IsWeekday,date-calendar-week,date-year
0,2017-12-08,Unknown,144.139.133.243,140,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
1,2017-12-08,Unknown,144.139.133.243,46,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
2,2017-12-08,Unknown,144.139.133.243,78,Desktop,Firefox,NaN,Melbourne,Australia,1,49,2017
3,2017-12-08,Unknown,144.139.133.243,171,Desktop,Firefox,Security,Melbourne,Australia,1,49,2017
4,2017-12-08,Unknown,144.139.133.243,46,Desktop,Firefox,Security,Melbourne,Australia,1,49,2017


#### 2- Automate load logs and aggregate data

In [ ]:
#df.to_csv('test.csv')

In [ ]:
client_browser_df = (df.groupby(by=['date','client-browser'])['client-browser']
                        .count()
                        .reset_index(level=1,name='Count')
                        .pivot(columns='client-browser',values='Count'))

client_city_df = (df.groupby(by=['date','client-city'])['client-city']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-city',values='Count'))

client_country_df = (df.groupby(by=['date','client-country'])['client-country']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-country',values='Count'))

client_device_df = (df.groupby(by=['date','client-device'])['client-device']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-device',values='Count'))

client_webPage_df = (df.groupby(by=['date','client-webPage'])['client-webPage']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-webPage',values='Count'))

(df.groupby(by=['date','client-ip'])['client-ip']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-ip',values='Count'))

In [ ]:
df.iloc[:,[0,5,6,11,12,13,14,15,16,17,18,19]]

In [ ]:
df.info()

In [ ]:
df["server-port"].describe()

## Derive Calendar Week, year after aggregating the data

In [ ]:
df['date-weekday']   = df['date'].apply(lambda x: 'Weekday' if np.int8(str(dt.datetime.strptime(x,'%Y-%m-%d').weekday())) < 5 else 'Weekend')
df['date-calendar-week']   = df['date'].apply(lambda x: np.int8(str(dt.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])))
df['date-year']   = df['date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d').year)

In [ ]:
df['date-weekday']   = df['date'].apply(lambda x: 1 if np.int8(str(dt.datetime.strptime(x,'%Y-%m-%d').weekday())) < 5 else 0)

In [ ]:
df.head()

In [ ]:
df.loc[:, df.dtypes == object].describe()

In [ ]:
import datetime as dt
d =dt.datetime.strptime(df.date.loc[1],'%Y-%m-%d')
d =dt.datetime.strptime('2018-12-28','%Y-%m-%d')
d.year
#np.int8(str(d.year))

#iis_logs_df.date.loc[1]

In [ ]:
df.isna().sum()